---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [199]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
# df.head(10)

# print(len(df))

# find all date in the format of / / or - - with only digits
# find all date in the format with month in letter
# advanced for the cases with st nd rd th
# case without date, only month
# case with only year

In [200]:
#     simple regression: full month, day, year in digit
exp = r'(?P<time>\d{1,2}[/-]\d{1,2}[/-]\d{2,4})'
df1 = df.str.extract(exp).dropna()
print(df1.shape)    # (126, 3)
# print(df1[100:150])
#     in this way, only 126 from 500 rows can be extracted. Let's improved our experession
# convert string to date time format
time_01 = pd.to_datetime(df1.time)

(125, 1)


In [201]:
df2 = df.drop(df1.index.tolist())
print(len(df2))
# full month, day, year but month is in 3 letters
exp2 = r'(?P<time>\d{1,2}[.\s-](?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec])[a-z]*[.\s-]\d{2,4})'
exp2 = r'(?P<time>\d{1,2}(?:st|nd|rd)?[.\s\t-]*(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec])[a-z]*[.\s\t-]*\d{2,4})'
df3a = df2.str.extract(exp2)
# handling exceptions
"""
174    .On 04 Dec 1988 the patient was admitted to Fa...
179    .On 10 Dec 1982, the first night after hospita...
180    .On 09 Dec 1988 the patient was discharged hom
"""
df3a.loc[174] = '4 Dec 1988'
df3a.loc[179] = '10 Dec 1982'
df3a.loc[180] = '9 Dec 1988'
df3 = df3a.dropna()
# convert string to datetime format
time_02 = pd.to_datetime(df3.time)

print(df3.shape)   # (66, 1)
df4 = df2.drop(df3.index.tolist())

375
(69, 1)


In [202]:
exp3 = r'(?P<time>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec])[a-z]*[.\t-,]\s*\d{1,2}(?:st|nd|rd)?[.\t-,]\s*\d{2,4})'
df5a = df4.str.extract(exp3)
# handling exception
df5a.loc[201] = 'December 23, 1999'
df5a.loc[219] = 'Dec 14 1975'
df5 = df5a.dropna()
# convert to time
time_03 = pd.to_datetime(df5.time)
# print(time_03)
df6 = df4.drop(df5.index.tolist())
print(df6.shape)

(272,)


In [203]:
# only month and year
exp4 = r'(?P<time>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec])[a-z]*[.\t-,]\s*\d{2,4})'
df7a = df6.str.extract(exp4).dropna()
df7a.loc[251] = 'Dec 1998'
df7a.loc[268] = 'Dec 2009'
df7a.loc[290] = 'Dec 2011'
df7a.loc[301] = 'Dec 1992'
df7a.loc[313] = 'Dec 1978'
df7 = df7a.dropna()
# handling exception Janaury
df7['time'] = df7['time'].str.replace('Janaury', 'January', regex=True)
print(len(df7))
# convert to time, the missing day is automatically filled by 1st day of the month
time_04 = pd.to_datetime(df7.time)

df8 = df6.drop(df7.index.tolist())
print(len(df8))

113
159


In [204]:
# only month and year digit
exp5 = r'(?P<time>\d{1,2}[/-]\s*\d{2,4})'
df9 = df8.str.extract(exp5).dropna()
# convert to datetime
time_05 = pd.to_datetime(df9.time)
# remaining data
df10 = df8.drop(df9.index.tolist())
print(df10.shape)

(47,)


In [221]:
exp6 = r'(?P<time>\d{4})'
df11 = df10.str.extract(exp6).dropna()
print(df11.shape)
# convert to datetime
time_06 = pd.to_datetime(df11.time)
# print(time_06)

(47, 1)


In [222]:
# merge all data frame
df_list = [time_01, time_02, time_03, time_04, time_05, time_06]
df_concat = pd.concat(df_list)
print(len(df_concat))
# since the len=500, it means I handled all exception

500


In [227]:
# sort the time
df_sorted = df_concat.sort_values(0)
# print(df_sorted[:50])
# create series
res = pd.Series(df_sorted.index.tolist())
print(res)

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     405
20     323
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    243
472    208
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    255
486    439
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64


In [207]:
def date_sorter():
    
    # note that df here is a series, not a data frame
    
    # length of each row
    # print(df.str.len())
    
    # check if each row contains digit number
    # note: .all() returns True if all element of the series are True
    # print(df.str.contains(r"\d").all())
    
    # count how many times the digits appear in each string
    # print(df.str.count(r'\d'))
    

    
    # full month, day, year but month is in 3 letters
    # exp2 = r'(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[/-|\s,.]+(?P<day>\d{1,2})[/-|\s,.](?P<year>\d{2,4})'
    # df2 = df.str.extractall(exp2)
    # print(df2.shape)   # (175, 3)
    # print(df2)

    # full month, day, year but month is in letters, can be 3 or longer
    #exp2 = r'(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[/-|\s,.]+(?P<day>\d{1,2})[/-|\s,.](?P<year>\d{2,4})'
    #df2 = df.str.extractall(exp2)
    #print(df2.shape)   # (175, 3)
    #print(df2)
    
    print(df.loc[125,])
    # add one more column with full information
    #exp3 = r'(?P<full_info>(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[/-|\s,.](?P<day>\d{1,2})[/-|\s,.](?P<year>\d{2,4}))'
    #df3 = df.str.extractall(exp3)
    #print(df3.shape)   # (175, 3)
    #print(df3)
    
    # day can be before or after month
    #exp4 = r'((?:\d{2} )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* (?:\d{2}, )?\d{2,4})'
    #exp4 = r'((\d{2}[/-|\s,.])?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[/-|\s,.](?:\d{2}[/-|\s,.])?(\d{2,4}))'
    #df4 = df.str.extractall(exp4)
    #print(df4.shape)   # (175, 3)
    #print(df4)
    
    # month before day
#     exp2 = r'(?P<time>(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[/-|\s,.])(?P<day>\d{1,2}[/|-|\s,.])(?P<year>\d{2,4}))'
#     df2 = df.str.extractall(exp2)
#     print(df2.shape)   # (5, 3)
#     print(df2)    

#     teststr = ['']

#     # month after day
#     exp3 = r'(?P<time>(?P<day>\d{1,2}[/|-|\s,.])(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[/|-|\s,.])(?P<year>\d{2,4}))'
#     df3 = df.str.extractall(exp3)
#     print(df3.shape)   # (69, 3)
#     print(df3)


    
    # only month and year


    
#     print(df.head())
#     print(list(df))
    return 'answer'

date_sorter()

s The patient is a 44 year old married Caucasian woman, unemployed Decorator, living with husband and caring for two young children, who is referred by Capitol Hill Hospital PCP, Dr. Heather Zubia, for urgent evaluation/treatment till first visit with Dr. Toney Winkler IN EIGHT WEEKS on 24 Jan 2001.



'answer'

In [3]:
175+ 126 + 69

370